In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
os.getenv("FSQCLID")[:3]

'MW4'

In [5]:
import json, requests

# I have to make a function to melts Geopoints into 4SQ `ll` format
ll ='40.7243,-74.0018'

def getCategoryFrom4SQ(ll, venue_category, radius='2000', limit=5):
    """
    A basic querying function that 
    INPUTS: 
       - the venue_category name, as defined by FOURSQUARE API.
    OUTPUT:
       - an array of GeoPoints that
    """
    #Using the `/venues/search` enpoint
    url = 'https://api.foursquare.com/v2/venues/search'

    if type(venue_category) is str:
        # Define a dictionary with the venue categories ID
        # https://developer.foursquare.com/docs/build-with-foursquare/categories/
        #
        # ♠ OPTIMIZATION IDEA: Scrape the official website to build the complete dictionary
        #    and save it in a separate categories.py that can also filter correct categories
        #    and also generate mixed categories (Meaning, getting 'international transport' 
        #    to include heliports and airports in just one search.)
        relevant_categories = {'daycare':         '5744ccdfe4b0c0459246b4c7',
                               
                               'coffee shop':    '4bf58dd8d48988d1e0931735',
                               'corporate coffee shop':'5665c7b9498e7d8a4f2c0f06', # More similar categories
                               
                               'airport':        '4bf58dd8d48988d1ed931735',
                               'heliport':       '56aa371ce4b08b9a8d57356e',
                               
                               'convention center':'4bf58dd8d48988d1ff931735',
                               'business center': '56aa371be4b08b9a8d573517',
                               
                               'design studio':'4bf58dd8d48988d1f4941735',
                               
                               'nightlife spot':'4d4b7105d754a06376d81259',
                               'arcade':       '4bf58dd8d48988d1e1931735',
                               
                               'volcano':      '5032848691d4c4b30a586d61',
                              }
        
        venue_category_ID = relevant_categories[venue_category]
    else: 
        raise TypeError('You must use one of the predefined keys from the relevant_categories dictionary.')    
    


    # Set API Query parameters
    params = dict(
        client_id=os.getenv('FSQCLID'),
        client_secret=os.getenv('FSQCLSC'),
        v='20190323',   # This `v` should not be changed, untill API is to be re-developed, 
                        #  see: https://developer.foursquare.com/docs/places-api/versioning/
        ll=ll, # format : '40.7243,-74.0018'
        query=None,
        categoryId=venue_category_ID,
        radius=radius,
        limit=limit
    )

    print(f"Requesting: {url},    venue_category: {venue_category}" )
    response = requests.get(url=url, params=params)
    
    print(response.status_code) #https://http.cat/

    return response

In [6]:
resp = getCategoryFrom4SQ(ll, 'coffee shop')

Requesting: https://api.foursquare.com/v2/venues/search,    venue_category: coffee shop
200


In [7]:
resp.json()['response'].keys()

dict_keys(['venues', 'confident'])

In [8]:
daycares = getCategoryFrom4SQ('40.7243,-74.0018', 'daycare')

Requesting: https://api.foursquare.com/v2/venues/search,    venue_category: daycare
200


In [9]:
for venue in daycares.json()['response']['venues']:
    print(venue['name'],'\n', venue['location'])

Bright Horizons at Tribeca 
 {'address': '129 Hudson St', 'lat': 40.720459810137804, 'lng': -74.00884164506351, 'labeledLatLngs': [{'label': 'display', 'lat': 40.720459810137804, 'lng': -74.00884164506351}, {'label': 'entrance', 'lat': 40.720415, 'lng': -74.008664}], 'distance': 731, 'postalCode': '10013', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['129 Hudson St', 'New York, NY 10013', 'United States']}
Bright Horizons in 90 Washington on West Street 
 {'address': '90 Washington St', 'lat': 40.70808107301849, 'lng': -74.0149106830358, 'labeledLatLngs': [{'label': 'display', 'lat': 40.70808107301849, 'lng': -74.0149106830358}], 'distance': 2117, 'postalCode': '10006', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['90 Washington St', 'New York, NY 10006', 'United States']}
Kiddie Academy of Hoboken 
 {'address': '112 Sinatra Drive', 'lat': 40.7376559, 'lng': -74.027726, 'labeledLatLngs

In [10]:
airports = getCategoryFrom4SQ('40.7243,-74.0018', 'airport', radius=30000)

Requesting: https://api.foursquare.com/v2/venues/search,    venue_category: airport
200


In [11]:
airports.json()

{'meta': {'code': 200, 'requestId': '5e9aeb3b3907e7001b7acb50'},
 'response': {'venues': [{'id': '4459d2f1f964a520d7321fe3',
    'name': 'Newark Liberty International Airport (EWR) (Newark Liberty International Airport)',
    'location': {'address': '10 Toler Pl',
     'crossStreet': 'at Express Rd',
     'lat': 40.692971458363075,
     'lng': -74.17799234390257,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.692971458363075,
       'lng': -74.17799234390257}],
     'distance': 15271,
     'postalCode': '07114',
     'cc': 'US',
     'city': 'Newark',
     'state': 'NJ',
     'country': 'United States',
     'formattedAddress': ['10 Toler Pl (at Express Rd)',
      'Newark, NJ 07114',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ed931735',
      'name': 'Airport',
      'pluralName': 'Airports',
      'shortName': 'Airport',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
       'suffix': '.png'},
      'primary':

In [12]:
nightlife_spots = getCategoryFrom4SQ('40.7243,-74.0018', 'nightlife spot', radius=500)

Requesting: https://api.foursquare.com/v2/venues/search,    venue_category: nightlife spot
200


In [14]:
nightlife_spots.json()

{'meta': {'code': 200, 'requestId': '5e9aebdc9fcb92001b5694a8'},
 'response': {'venues': [{'id': '5e88c7acb223d800080dce56',
    'name': 'Torch & Crown Brewing Company',
    'location': {'address': '12 Vandam St',
     'lat': 40.726314,
     'lng': -74.004208,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.726314,
       'lng': -74.004208}],
     'distance': 302,
     'postalCode': '10013',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['12 Vandam St',
      'New York, NY 10013',
      'United States']},
    'categories': [{'id': '50327c8591d4c4b30a586d5d',
      'name': 'Brewery',
      'pluralName': 'Breweries',
      'shortName': 'Brewery',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/brewery_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587211191',
    'hasPerk': False},
   {'id': '5c44c91e75eee4002c0a4ca9',
    'name': 'Niche Niche',
    '

In [ ]:
data = json.loads(res.c)
data